In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import random
import re
import joblib
from rdkit.Chem import QED

In [2]:
df = pd.read_csv('./n2_pred.csv')

In [3]:
mask = df['prob']>0.7

In [4]:
df[mask]

,cid,iupac,desc,smiles,pred,prob
35,65983216,3-[[N-methyl-2-(propylaminomethyl)anilino]meth...,"The molecule contains methyl, propylaminomethy...",CCCNCc1ccccc1N(C)CC1CC(O)C1,1.0,0.765831
57,17173343,"2,3-dichloro-N-(2-methoxyethyl)benzamide","The molecule contains dichloro, methoxyethyl, ...",COCCNC(=O)c1cccc(Cl)c1Cl,1.0,0.787722
87,140960198,"2,2,3,3,5,5,6,6-octadeuterio-1-(4-nitrophenyl)...","The molecule contains octadeuterio, nitropheny...",O=[N+]([O-])c1ccc(N2CCNCC2)cc1,1.0,0.807710
119,43267355,N-cyclopentyl-N-methyl-2-piperidin-4-ylacetamide,"The molecule contains cyclopentyl, methyl, pip...",CN(C(=O)CC1CCNCC1)C1CCCC1,1.0,0.745748
136,115648108,"2-[1-[(5-methyl-1,2,4-oxadiazol-3-yl)methyl]py...","The molecule contains methyl, oxadiazol, yl, m...",Cc1nc(CN2CCC(CCO)C2)no1,1.0,0.938234
...,...,...,...,...,...,...
499870,20240398,N-hydroxy-N-methylhex-5-enamide,"The molecule contains hydroxy, methylhex, enam...",C=CCCCC(=O)N(C)O,1.0,0.783318
499912,113841679,[4-(oxolan-2-ylmethylsulfinylmethyl)pyridin-2-...,"The molecule contains oxolan, ylmethylsulfinyl...",NCc1cc(CS(=O)CC2CCCO2)ccn1,1.0,0.766039
499919,129484260,"5-methyl-4-[(3S)-4-methyl-3-(3-methyl-1,2,4-ox...","The molecule contains methyl, methyl, methyl, ...",Cc1noc(C2CN(C(=O)c3cc(C#N)oc3C)CCN2C)n1,1.0,0.904013
499965,80256997,"3-(chloromethyl)-5-(1-methylcyclopentyl)-1,2,4...","The molecule contains chloromethyl, methylcycl...",CC1(c2nc(CCl)no2)CCCC1,1.0,0.853734


In [29]:
outputs = []
for item in df[mask].values:
    smiles = item[3]
    iupac = item[1]

    m=Chem.MolFromSmiles(smiles)
    qed=QED.qed(m)               # QED analysis

    word_list = re.split("[\s\[\],\(\)-.;]",iupac)
    # filtered_word_list = [item for item in word_list if len(item)>1 and item[0].isnumeric() is False]
    filtered_word_list = [item for item in word_list if len(item)>2 and item[0].isnumeric() is False]
    if len(filtered_word_list) == 0:
        continue

    # used_fg = []
    for fg in filtered_word_list:
        # if fg in used_fg:
        #     continue
        
        desc = f"The molecule contains {fg}. It can bind to 3CL."
        outputs.append([0,0,desc,smiles])

        
        if qed > 0.6:
            desc2 = desc + ' It has a high qed score.'
        else:
            desc2 = desc + ' It has a low qed score.'

        outputs.append([0,iupac,desc2,smiles])

        # used_fg.append(fg)

In [30]:
outputs = pd.DataFrame(data=outputs, columns=['cid', 'iupac', 'desc', 'smiles'])
outputs.to_csv(f'../data/train/3CL.csv', index=False)
print('Saved.')

Saved.
